# Recommender Systems with DGL

## Introduction

Graph Neural Networks (GNN), as a methodology of learning representations on graphs, has gained much attention recently.  Various models such as Graph Convolutional Networks, GraphSAGE, etc. are proposed to obtain representations of whole graphs, or nodes on a single graph.

A primary goal of recommendation is to automatically make predictions about a user's interest, e.g. whether/how a user would interact with a set of items, given the interaction history of the user herself, as well as the histories of other users.  The user-item interaction can also be viewed as a bipartite graph, where users and items form two sets of nodes, and edges connecting them stands for interactions.  The problem can then be formulated as a *link-prediction* problem, where we try to predict whether an edge (of a given type) exists between two nodes.

Based on this intuition, the academia developed multiple new models for recommendation, including but not limited to:

* Geometric Learning Approaches
  * [Geometric Matrix Completion](https://papers.nips.cc/paper/5938-collaborative-filtering-with-graph-information-consistency-and-scalable-methods.pdf)
  * [Recurrent Multi-graph CNN](https://arxiv.org/pdf/1704.06803.pdf)
* Graph-convolutional Approaches
  * Models such as [R-GCN](https://arxiv.org/pdf/1703.06103.pdf) or [GraphSAGE](https://github.com/stellargraph/stellargraph/tree/develop/demos/link-prediction/hinsage) also apply.
  * [Graph Convolutional Matrix Completion](https://arxiv.org/abs/1706.02263)
  * [PinSage](https://arxiv.org/pdf/1806.01973.pdf)
  
In this hands-on tutorial, we will demonstrate how to write a recommendation model with GraphSAGE in DGL + MXNet. We just demonstrate rating prediction in this tutorial.

In [ ]:
!pip install dgl==0.4.3

     |████████████████████████████████| 3.0MB 2.8MB/s 


In [ ]:
!pip install mxnet

     |████████████████████████████████| 55.0MB 74kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
import dgl
import dgl.function as FN

# Load MXNet as backend
dgl.load_backend('mxnet')

In [ ]:
import mxnet as mx
from mxnet import ndarray as nd, autograd, gluon
from mxnet.gluon import nn
import numpy as np

np.random.seed(0)

## Loading data

In this tutorial, we focus on rating prediction on MovieLens-100K dataset.  The data comes from [MovieLens](http://files.grouplens.org/datasets/movielens/ml-1m.zip) and is shipped with the notebook already.

The dataset is encapsulated in `movielens.MovieLens` class for clarity of this notebook.

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import string
from functools import partial

class MovieLens(object):
    split_by_time = None
    
    def read_user_line(self, l):
        id_, age, gender, occupation, zip_ = l.strip().split('|')
        age = np.searchsorted([20, 30, 40, 50, 60], age)   # bin the ages into <20, 20-30, 30-40, ..., >60
        return {'id': int(id_), 'gender': gender, 'age': age, 'occupation': occupation, 'zip': zip_}
    
    def read_product_line(self, l):
        fields = l.strip().split('|')
        id_ = fields[0]
        title = fields[1]
        genres = fields[-19:]

        # extract year
        if re.match(r'.*\([0-9]{4}\)$', title):
            year = title[-5:-1]
            title = title[:-6].strip()
        else:
            year = 0

        data = {'id': int(id_), 'title': title, 'year': year}
        for i, g in enumerate(genres):
            g = int(g)
            data['genre' + str(i)] = (g != 0)
        return data
    
    def read_rating_line(self, l):
        user_id, product_id, rating, timestamp = l.split()
        return {'user_id': int(user_id), 'product_id': int(product_id), 'rating': float(rating), 'timestamp': int(timestamp)}
    
    def __init__(self, directory):
        '''
        directory: path to movielens directory which should have the three
                   files:
                   users.dat
                   products.dat
                   ratings.dat
        '''
        self.directory = directory

        users = []
        products = []
        ratings = []

        # read ratings
        with open(os.path.join(directory, 'ua.base')) as f:
            for l in f:
                rating = self.read_rating_line(l)
                ratings.append(rating)
        with open(os.path.join(directory, 'ua.test')) as f:
            for l in f:
                rating = self.read_rating_line(l)
                ratings.append(rating)
                
        ratings = pd.DataFrame(ratings)
        product_count = ratings['product_id'].value_counts()
        product_count.name = 'product_count'
        ratings = ratings.join(product_count, on='product_id')
        self.ratings = ratings

        # read users - if user feature does not exist, we find all unique user IDs
        # appeared in the rating table and create an empty table from that.
        user_file = os.path.join(directory, 'u.user')
        with open(user_file) as f:
            for l in f:
                users.append(self.read_user_line(l))
        self.users = pd.DataFrame(users).set_index('id').astype('category')

        # read products
        with open(os.path.join(directory, 'u.item'), encoding='latin1') as f:
            for l in f:
                products.append(self.read_product_line(l))
        self.products = (
                pd.DataFrame(products)
                .set_index('id')
                .astype({'year': 'category'}))
        self.genres = self.products.columns[self.products.dtypes == bool]
        
        self.ratings = self.data_split(self.ratings)
        self.users = self.users[self.users.index.isin(self.ratings['user_id'])]
        self.products = self.products[self.products.index.isin(self.ratings['product_id'])]
        
        self.build_graph()
        self.generate_mask()
        self.generate_candidates()
        
        # Construct the training subgraph.
        train_eid = self.g.filter_edges(lambda edges: edges.data['train']).astype('int64')
        self.g_train = self.g.edge_subgraph(train_eid, preserve_nodes=True)
        self.g_train.copy_from_parent()

    # Change this field to 'timestamp' to perform a user-based temporal split of the dataset,
    # where the ratings of each users are ordered by timestamps.  The first 80% of the ordered
    # data falls into the training set, and the middle 10% and last 10% goes to validation and
    # test set respectively.
    # If this field is None, the ratings of each users would be shuffled and splitted in a
    # ratio of 80-10-10.
    split_by_time = None
    def split_user(self, df, filter_counts=0, timestamp=None):
        df_new = df.copy()
        df_new['prob'] = -1

        df_new_sub = (df_new['product_count'] >= filter_counts).to_numpy().nonzero()[0]
        prob = np.linspace(0, 1, df_new_sub.shape[0], endpoint=False)
        if timestamp is not None and timestamp in df_new.columns:
            df_new = df_new.iloc[df_new_sub].sort_values(timestamp)
            df_new['prob'] = prob
            return df_new
        else:
            np.random.shuffle(prob)
            df_new['prob'].iloc[df_new_sub] = prob
            return df_new

    def data_split(self, ratings):
        ratings = ratings.groupby('user_id', group_keys=False).apply(
                partial(self.split_user, filter_counts=10, timestamp=self.split_by_time))
        ratings['train'] = ratings['prob'] <= 0.8
        ratings['valid'] = (ratings['prob'] > 0.8) & (ratings['prob'] <= 0.8)
        ratings['test'] = ratings['prob'] > 0.8
        ratings.drop(['prob'], axis=1, inplace=True)
        return ratings


    # process the features and build the DGL graph
    def build_graph(self):
        import mxnet as mx
        from mxnet import ndarray as nd
        from mxnet import gluon, autograd
        import dgl
        
        user_ids = list(self.users.index)
        product_ids = list(self.products.index)
        user_ids_invmap = {id_: i for i, id_ in enumerate(user_ids)}
        product_ids_invmap = {id_: i for i, id_ in enumerate(product_ids)}
        self.user_ids = user_ids
        self.product_ids = product_ids
        self.user_ids_invmap = user_ids_invmap
        self.product_ids_invmap = product_ids_invmap

        g = dgl.DGLGraph(multigraph=True)
        g.add_nodes(len(user_ids) + len(product_ids))

        # node type
        node_type = nd.zeros(g.number_of_nodes(), dtype='float32')
        node_type[:len(user_ids)] = 1
        g.ndata['type'] = node_type

        # user features
        print('Adding user features...')
        for user_column in self.users.columns:
            udata = nd.zeros(g.number_of_nodes(), dtype='int64')
            # 0 for padding
            udata[:len(user_ids)] = \
                    nd.from_numpy(self.users[user_column].cat.codes.values.astype('int64') + 1)
            g.ndata[user_column] = udata

        # product genre
        print('Adding product features...')
        product_genres = nd.from_numpy(self.products[self.genres].values.copy().astype('float32'))
        g.ndata['genre'] = nd.zeros((g.number_of_nodes(), len(self.genres)))
        g.ndata['genre'][len(user_ids):len(user_ids) + len(product_ids)] = product_genres

        # product year
        if 'year' in self.products.columns:
            g.ndata['year'] = nd.zeros(g.number_of_nodes(), dtype='int64')
            # 0 for padding
            g.ndata['year'][len(user_ids):len(user_ids) + len(product_ids)] = \
                    nd.from_numpy(self.products['year'].cat.codes.values.astype('int64') + 1)

        '''
        # product title
        print('Parsing title...')
        nlp = stanfordnlp.Pipeline(use_gpu=False, processors='tokenize,lemma')
        vocab = set()
        title_words = []
        for t in tqdm.tqdm(self.products['title'].values):
            doc = nlp(t)
            words = set()
            for s in doc.sentences:
                words.update(w.lemma.lower() for w in s.words
                             if not re.fullmatch(r'['+string.punctuation+']+', w.lemma))
            vocab.update(words)
            title_words.append(words)
        vocab = list(vocab)
        vocab_invmap = {w: i for i, w in enumerate(vocab)}
        # bag-of-words
        g.ndata['title'] = nd.zeros((g.number_of_nodes(), len(vocab)))
        for i, tw in enumerate(tqdm.tqdm(title_words)):
            g.ndata['title'][len(user_ids) + i, [vocab_invmap[w] for w in tw]] = 1
        self.vocab = vocab
        self.vocab_invmap = vocab_invmap
        '''
        
        rating_user_vertices = [user_ids_invmap[id_] for id_ in self.ratings['user_id'].values]
        rating_product_vertices = [product_ids_invmap[id_] + len(user_ids)
                                 for id_ in self.ratings['product_id'].values]
        self.rating_user_vertices = rating_user_vertices
        self.rating_product_vertices = rating_product_vertices

        g.add_edges(
                rating_user_vertices,
                rating_product_vertices,
                data={'inv': nd.zeros(self.ratings.shape[0], dtype='int32'),
                    'rating': nd.from_numpy(self.ratings['rating'].values.astype('float32'))})
        g.add_edges(
                rating_product_vertices,
                rating_user_vertices,
                data={'inv': nd.ones(self.ratings.shape[0], dtype='int32'),
                    'rating': nd.from_numpy(self.ratings['rating'].values.astype('float32'))})
        self.g = g
        g.readonly()
        
    # Assign masks of training, validation and test set onto the DGL graph
    # according to the rating table.
    def generate_mask(self):
        from mxnet import ndarray as nd
        valid_tensor = nd.from_numpy(self.ratings['valid'].values.astype('int32'))
        test_tensor = nd.from_numpy(self.ratings['test'].values.astype('int32'))
        train_tensor = nd.from_numpy(self.ratings['train'].values.astype('int32'))
        edge_data = {
                'valid': valid_tensor,
                'test': test_tensor,
                'train': train_tensor,
                }

        self.g.edges[self.rating_user_vertices, self.rating_product_vertices].data.update(edge_data)
        self.g.edges[self.rating_product_vertices, self.rating_user_vertices].data.update(edge_data)
        
    # Generate the list of products for each user in training/validation/test set.
    def generate_candidates(self):
        self.p_train = []
        self.p_valid = []
        self.p_test = []
        for uid in self.user_ids:
            user_ratings = self.ratings[self.ratings['user_id'] == uid]
            self.p_train.append(np.array(
                [self.product_ids_invmap[i] for i in user_ratings[user_ratings['train']]['product_id'].values]))
            self.p_valid.append(np.array(
                [self.product_ids_invmap[i] for i in user_ratings[user_ratings['valid']]['product_id'].values]))
            self.p_test.append(np.array(
                [self.product_ids_invmap[i] for i in user_ratings[user_ratings['test']]['product_id'].values]))

In [ ]:
!git clone https://github.com/dglai/KDD-2019-Hands-on.git

Cloning into 'KDD-2019-Hands-on'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 273 (delta 3), reused 0 (delta 0), pack-reused 265
Receiving objects: 100% (273/273), 19.78 MiB | 22.86 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
ml = MovieLens('/content/KDD-2019-Hands-on/3_recommender_system/ml-100k')

/usr/local/lib/python3.6/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)


Adding user features...
Adding product features...


The MovieLens graph has 2,625 nodes and 200,000 edges. The graph contains both user nodes and movie nodes. The MovieLens dataset contains 100,000 ratings and the ratings are stored as edges. DGL only supports directed edges. Thus, each rating is stored twice: one connects from a user node to a movie node, and the other connects from a movie node to a user node.

In [ ]:
# This is a DGLGraph object.
g = ml.g
print('#vertices:', g.number_of_nodes())
print('#edges:', g.number_of_edges())

#vertices: 2625
#edges: 200000


## See the features in the MovieLens dataset

The MovieLens dataset has some user features and movie features.

User features:
* age,
* gender,
* occupation,
* zip code,

Movie features:
* genre,
* year,

We use one-hot encoding for "age", "gender", "occupation", "zip code" and "year". "genre" uses multi-hot encoding. We then store them as node features on the graph.

Since user features and item features are different, we pad both types of features with zeros.

In [ ]:
print(g.ndata)

{'type': 
[1. 1. 1. ... 0. 0. 0.]
<NDArray 2625 @cpu(0)>, 'gender': 
[2 1 2 ... 0 0 0]
<NDArray 2625 @cpu(0)>, 'age': 
[2 5 2 ... 0 0 0]
<NDArray 2625 @cpu(0)>, 'occupation': 
[20 14 21 ...  0  0  0]
<NDArray 2625 @cpu(0)>, 'zip': 
[623 690 271 ...   0   0   0]
<NDArray 2625 @cpu(0)>, 'genre': 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 2625x19 @cpu(0)>, 'year': 
[ 0  0  0 ... 72 68 65]
<NDArray 2625 @cpu(0)>}


There is a node data "type" that indicates the node type in the bipartite graph. Nodes with "type=1" are user nodes and "type=0" are movie nodes.

In [ ]:
print('#user nodes:', mx.nd.sum(g.ndata['type'] == 1).asnumpy())
print('#movie nodes:', mx.nd.sum(g.ndata['type'] == 0).asnumpy())

#user nodes: [943.]
#movie nodes: [1682.]


# Recommendation model with GNN

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/rec_process.png" width="800">

Recommendation with graph neural networks has two steps:
* graph encoder: use graph neural networks to compute node embeddings.
* edge decoder: compute scores on edges with user embeddings and movie embeddings.

In [ ]:
class GNNRecommender(nn.Block):
    def __init__(self, encoder, decoder):
        super(GNNRecommender, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, G, users, items):
        h = self.encoder(G)
        h_users = h[users]
        h_items = h[items]
        return self.decoder(h_users, h_items, users, items)

## GraphSage encoder

The encoder does two things:
* generate the initial user and movie embeddings from categorial features.
* run GraphSAGE layers on the MovieLens graph multiple times to compute the final embeddings for rating prediction.

In [ ]:
class GraphSageEncoder(nn.Block):
    def __init__(self, embedding_size, n_layers, G):
        super(GraphSageEncoder, self).__init__()

        self.G = G
        self.user_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 1).astype(np.int64)
        self.movie_nodes = G.filter_nodes(lambda nodes: nodes.data['type'] == 0).astype(np.int64)

        self.layers = nn.Sequential()
        for i in range(n_layers):
            self.layers.add(GraphSageLayer(embedding_size, G, self.user_nodes, self.movie_nodes))

        # One-hot encoding for each node.
        node_emb = nn.Embedding(G.number_of_nodes() + 1, embedding_size)
        self.user_emb = UserEmbedding(G, embedding_size, node_emb)
        self.movie_emb = MovieEmbedding(G, embedding_size, node_emb)

    def forward(self, G):
        # Generate embeddings on user nodes and movie nodes.
        assert G.number_of_nodes() == self.G.number_of_nodes()
        G.apply_nodes(lambda nodes: {'h': self.user_emb(nodes.data, self.user_nodes)}, self.user_nodes)
        G.apply_nodes(lambda nodes: {'h': self.movie_emb(nodes.data, self.movie_nodes)}, self.movie_nodes)

        for layer in self.layers:
            layer(G)

        # The node embeddings computed by GraphSage layers are stored in node data of 'h'.
        return G.ndata['h']

## GraphSage model

We can now write a GraphSAGE layer.  In GraphSAGE, the node representation is updated with the representation in the previous layer as well as an aggregation (often mean) of "messages" sent from all neighboring nodes.

### Algorithm

The algorithm of a single GraphSAGE layer has three steps for each node $v$:

1. $h_{\mathcal{N}(v)} \gets \mathtt{Sum}_{u \in \mathcal{N}(v)} h_{u}$
2. $h_{v} \gets \sigma\left(W \cdot \mathtt{CONCAT}(h_v, h_{\mathcal{N}(v)}/d_{\mathcal{N}(v)})\right)$
3. $h_{v} \gets h_{v} / \lVert h_{v} \rVert_2$

### Slight modification on the original GraphSage model

In practice, the MovieLens dataset has two types of nodes: users and movies. We need to perform separate node update functions on the two types of nodes on step 2.

For the movie nodes,

2. $h_{m} \gets \sigma\left(W0 \cdot \mathtt{CONCAT}(h_m, h_{\mathcal{N}(m)} / d_{\mathcal{N}(m)})\right)$

For the user nodes,

2. $h_{u} \gets \sigma\left(W1 \cdot \mathtt{CONCAT}(h_u, h_{\mathcal{N}(u)} / d_{\mathcal{N}(u)})\right)$

In [ ]:
class GraphSageLayer(nn.Block):
    def __init__(self, feature_size, G, user_nodes, movie_nodes):
        super(GraphSageLayer, self).__init__()

        self.feature_size = feature_size
        self.G = G
        self.user_nodes = user_nodes
        self.movie_nodes = movie_nodes

        self.user_update = GraphSageNodeUpdate(feature_size)
        self.movie_update = GraphSageNodeUpdate(feature_size)

        all_nodes = mx.nd.arange(G.number_of_nodes(), dtype=np.int64)
        self.deg = G.in_degrees(all_nodes).astype(np.float32)

    def forward(self, G):
        assert G.number_of_nodes() == self.G.number_of_nodes()
        G.ndata['deg'] = self.deg
        # Step 1
        # The message function and message reduce function used by GraphSage.
        # EXERCISE: Interested users can try different message functions and reduce functions.
        G.update_all(FN.copy_src('h', 'h'), FN.sum('h', 'h_agg'))
        # Step 2 and 3
        G.apply_nodes(self.user_update, self.user_nodes)
        G.apply_nodes(self.movie_update, self.movie_nodes)

In [ ]:
class GraphSageNodeUpdate(nn.Block):
    def __init__(self, feature_size):
        super(GraphSageNodeUpdate, self).__init__()

        self.feature_size = feature_size
        self.W = nn.Dense(feature_size)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, nodes):
        # Node embedding from the previous layer.
        h = nodes.data['h']
        # Aggregation of the node embeddings in the neighborhood
        h_agg = nodes.data['h_agg']
        # Degree of the vertex.
        deg = nodes.data['deg'].expand_dims(1)
        h_concat = nd.concat(h, h_agg / nd.maximum(deg, 1e-6), dim=1)
        h_new = self.leaky_relu(self.W(h_concat))
        # Layer norm
        return {'h': h_new / nd.maximum(h_new.norm(axis=1, keepdims=True), 1e-6)}

## Compute node embeddings on the MovieLens dataset

User nodes and movie nodes have different sets of features. Thus, we need to generate embeddings differently.

User nodes have categorial features of "age", "gender", "occupation" and "zip code". These features are all one-hot encodings. In addition, we add one-hot encoding for every user node. To generate user embedding, we add all of these embeddings together.

In [ ]:
class UserEmbedding(nn.Block):
    def __init__(self, G, feature_size, node_emb):
        super(UserEmbedding, self).__init__()

        # Embedding matrices for one-hot encoding.
        self.emb_age = nn.Embedding(G.ndata['age'].max().asscalar() + 1,
                                    feature_size)
        self.emb_gender = nn.Embedding(G.ndata['gender'].max().asscalar() + 1,
                                       feature_size)
        self.emb_occupation = nn.Embedding(G.ndata['occupation'].max().asscalar() + 1,
                                           feature_size)
        self.emb_zip = nn.Embedding(G.ndata['zip'].max().asscalar() + 1,
                                    feature_size)
        
        # One-hot encoding for each node.
        self.node_emb = node_emb

    def forward(self, ndata, nid):
        h = self.node_emb(nid + 1)
        extra_repr = []
        extra_repr.append(self.emb_age(ndata['age']))
        extra_repr.append(self.emb_gender(ndata['gender']))
        extra_repr.append(self.emb_occupation(ndata['occupation']))
        extra_repr.append(self.emb_zip(ndata['zip']))
        return h + nd.stack(*extra_repr, axis=0).sum(axis=0)

Movie nodes "year", "genre". "year" is one-hot encoding, "genre" is stored in a float32 dense matrix. Like user nodes, we add one-hot encoding to every movie node. To generate movie embedding, we add all of these embeddings together.

In [ ]:
class MovieEmbedding(nn.Block):
    def __init__(self, G, feature_size, node_emb):
        super(MovieEmbedding, self).__init__()
        self.emb_year = nn.Embedding(G.ndata['year'].max().asscalar() + 1,
                                     feature_size)

        # Linear projection for float32 features.
        seq = nn.Sequential()
        with seq.name_scope():
            seq.add(nn.Dense(feature_size))
            seq.add(nn.LeakyReLU(0.1))
        self.proj_genre = seq

        # One-hot encoding for each node.
        self.node_emb = node_emb

    def forward(self, ndata, nid):
        h = self.node_emb(nid + 1)
        extra_repr = []
        extra_repr.append(self.emb_year(ndata['year']))
        extra_repr.append(self.proj_genre(ndata['genre']))
        return h + nd.stack(*extra_repr, axis=0).sum(axis=0)

## Rating prediction

For recommendation, the rating on item $j$ by user $i$ is defined by $u_i^T v_j$.

In practice, recommendation models have user bias term and movie bias term: $u_i^T v_j + b_{u_i} + b_{v_j}$.

In [ ]:
class DotDecoder(nn.Block):
    def __init__(self, num_nodes):
        super(DotDecoder, self).__init__()
        
        with self.name_scope():
            self.biases = self.params.get(
                'node_biases',
                init=mx.init.Zero(),
                shape=(num_nodes+1,))
            
    def forward(self, h_users, h_items, users, items):
        user_biases = self.biases.data()[users+1]
        item_biases = self.biases.data()[items+1]
        return (h_users * h_items).sum(1) + user_biases + item_biases

## Get the training set

We use 80% of edges for training.

In [ ]:
g_train = ml.g_train
rating_train = g_train.edata['rating']
src_train, dst_train = g_train.all_edges()
print('#vertices:', g_train.number_of_nodes())
print('#edges:', g_train.number_of_edges())

#vertices: 2625
#edges: 161940


## Get the testing edge set

We use 20% of edges for evaluation.

In [ ]:
g = ml.g
eid_test = g.filter_edges(lambda edges: edges.data['test']).astype('int64')
src_test, dst_test = g.find_edges(eid_test)
rating_test = g.edges[eid_test].data['rating']
print('#testing edges:', len(eid_test))

#testing edges: 38060


## Run the model

In [ ]:
# We use 1-layer GraphSage as the graph encoder.
# EXERCISE: Interested users can try different numbers of layers (e.g., 0, 1, 2)
model = GNNRecommender(GraphSageEncoder(embedding_size=100, n_layers=1, G=g_train),
                       DotDecoder(g_train.number_of_nodes()))
model.collect_params().initialize(ctx=mx.cpu())
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.003, 'wd': 1e-5})

for epoch in range(200):
    # Training
    for _ in range(10):
        with mx.autograd.record():
            score = model(g_train, src_train, dst_train)
            loss = ((score - rating_train) ** 2).mean()
            loss.backward()
        trainer.step(1)

    # Testing
    h = model.encoder(g)
    # Compute test RMSE
    score = model.decoder(h[src_test], h[dst_test], src_test, dst_test)
    test_rmse = nd.sqrt(((score - rating_test) ** 2).mean())

    print('Training loss:', loss.asscalar(), 'Test RMSE:', test_rmse.asscalar())

Training loss: 7.425549 Test RMSE: 2.7286336
Training loss: 7.1118746 Test RMSE: 2.6723719


KeyboardInterrupt: ignored

In [ ]:
if 'h' in g_train.ndata:
    del g_train.ndata['h']
if 'h_agg' in g_train.ndata:
    del g_train.ndata['h_agg']
if 'h' in g.ndata:
    del g.ndata['h']
if 'h_agg' in g.ndata:
    del g.ndata['h_agg']

# Take-home Exercise

Interested users can try different things given this simple recommendation model with GraphSage. Interesting experiments include:
* try different numbers of GraphSage layers.
* try different graph encoders. For example, replace the message passing functions with the one in [GAT](https://github.com/dmlc/dgl/blob/master/examples/mxnet/gat/gat.py) or GCMC.
* try mini-batch training. A tutorial of training GraphSage with mini-batch has been included in the same folder of this tutorial.
* try different recommendation datasets. To explore the benefit of graph neural networks, we suggest users to try feature-rich datasets.